In [1]:
# autoreload
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pandas as pd
from tqdm import tqdm

In [4]:
mimic_iv_notes_parent = "/data/wang/junh/datasets/physionet.org/files/mimic-iv-note/2.2/note"
mimic_iv_path = "/data/wang/junh/datasets/physionet.org/files/mimiciv/2.2"

rad_notes_f_path = os.path.join(mimic_iv_notes_parent, "radiology.csv.gz")
rad_notes_df = pd.read_csv(rad_notes_f_path, low_memory=False)
rad_notes_df['charttime'] = pd.to_datetime(rad_notes_df['charttime'])
rad_notes_df['storetime'] = pd.to_datetime(rad_notes_df['storetime'])

icustays_df = pd.read_csv(os.path.join(mimic_iv_path, "icu", "icustays.csv.gz"), low_memory=False)
icustays_df['intime'] = pd.to_datetime(icustays_df['intime'])
icustays_df['outtime'] = pd.to_datetime(icustays_df['outtime'])

admissions_df = pd.read_csv(os.path.join(mimic_iv_path, "hosp", "admissions.csv.gz"), low_memory=False)
admissions_df['admittime'] = pd.to_datetime(admissions_df['admittime'])
admissions_df['dischtime'] = pd.to_datetime(admissions_df['dischtime'])

In [6]:
rad_notes_df['hadm_id'] = None
rad_notes_df['stay_id'] = None
rad_notes_df['icu_time_delta'] = None
rad_notes_df['hosp_time_delta'] = None

def calc_time_delta_hrs(icu_intime, charttime):
    return (charttime - icu_intime).total_seconds() / 3600

for index, row in tqdm(rad_notes_df.iterrows(), total=rad_notes_df.shape[0]):
    curr_pts_icustays = icustays_df[icustays_df['subject_id'] == row['subject_id']]
    
    for icu_index, icu_row in curr_pts_icustays.iterrows():
        if icu_row['intime'] <= row['charttime'] <= icu_row['outtime']:
            rad_notes_df.loc[index, 'stay_id'] = icu_row['stay_id']
            rad_notes_df.loc[index, 'icu_time_delta'] = calc_time_delta_hrs(icu_row['intime'], row['charttime'])
    
    curr_pts_admissions = admissions_df[admissions_df['subject_id'] == row['subject_id']]

    for hosp_index, hosp_row in curr_pts_admissions.iterrows():
        if hosp_row['admittime'] <= row['charttime'] <= hosp_row['dischtime']:
            rad_notes_df.loc[index, 'hadm_id'] = hosp_row['hadm_id']
            rad_notes_df.loc[index, 'hosp_time_delta'] = calc_time_delta_hrs(hosp_row['admittime'], row['charttime'])

  0%|          | 0/2321355 [00:00<?, ?it/s]

100%|██████████| 2321355/2321355 [42:41<00:00, 906.40it/s] 


In [5]:
rad_notes_df['hadm_id'] = None
rad_notes_df['stay_id'] = None
rad_notes_df['icu_time_delta'] = None
rad_notes_df['hosp_time_delta'] = None

def calc_time_delta_hrs(icu_intime, charttime):
    return (charttime - icu_intime).total_seconds() / 3600

for index, row in tqdm(rad_notes_df.iterrows(), total=rad_notes_df.shape[0]):
    curr_pts_icustays = icustays_df[icustays_df['subject_id'] == row['subject_id']]
    
    for icu_index, icu_row in curr_pts_icustays.iterrows():
        if icu_row['intime'] <= row['charttime'] <= icu_row['outtime']:
            rad_notes_df.loc[index, 'stay_id'] = icu_row['stay_id']
            rad_notes_df.loc[index, 'icu_time_delta'] = calc_time_delta_hrs(icu_row['intime'], row['charttime'])
    
    curr_pts_admissions = admissions_df[admissions_df['subject_id'] == row['subject_id']]

    for hosp_index, hosp_row in curr_pts_admissions.iterrows():
        if hosp_row['admittime'] <= row['charttime'] <= hosp_row['dischtime']:
            rad_notes_df.loc[index, 'hadm_id'] = hosp_row['hadm_id']
            rad_notes_df.loc[index, 'hosp_time_delta'] = calc_time_delta_hrs(hosp_row['admittime'], row['charttime'])

100%|██████████| 2321355/2321355 [40:02<00:00, 966.32it/s] 


In [6]:
mm_dir = "/data/wang/junh/datasets/multimodal"
output_dir = os.path.join(mm_dir, "preprocessing")

rad_notes_df.to_pickle(os.path.join(output_dir, "notes_text.pkl"))
print(rad_notes_df.shape)

(2321355, 11)


In [1]:
rad_notes_df

NameError: name 'rad_notes_df' is not defined